In [29]:
import numpy as np
import matplotlib.pyplot as plt

In [30]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [31]:
# generate SDR

out_len = 128 # 128
sparsity = 4 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / out_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], out_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for inp_SDR in letter_SDRs.values():
            if (inp_SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, inp_SDR in letter_SDRs.items():
    print(f'{letter} : {(inp_SDR > 0).sum()} {inp_SDR.tolist()}')

a : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
b : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
c : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [33]:
# init start column
column_size = 16

start_SDR = letter_SDRs['<']

start_I = np.zeros((start_SDR.size, column_size))

active_start_SDR_idx = np.where(start_SDR == 1)

for as_idx in active_start_SDR_idx[0]:
    rnd_active = np.random.randint(column_size)

    start_I[as_idx][rnd_active] = 1

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')

In [34]:
from numpy import ndarray


def train(input_SDRs: list[ndarray],
          push_delta=0.04,
          epoch_cnt=1,
          inp_len=128,
          out_len=128,
          sparsity=4,
          eps0=0.01,
          eps0_V=0.09,
          ):
    I = np.zeros(inp_len)
    K = np.zeros(out_len)

    S = np.random.rand(out_len, out_len) * 1e-2

    V = np.zeros(out_len)

    prec = 1e-30

    inputs_idx = np.arange(len(input_SDRs))

    for epoch_idx in range(epoch_cnt):
        np.random.shuffle(inputs_idx)

        # eps = (epoch_cnt - epoch_idx) * eps0

        for SDR_idx in inputs_idx:
            I = np.copy(input_SDRs[SDR_idx])

            K = np.dot(S, I)

            K /= np.max(np.abs(K)) + prec

            nV = np.abs(V - np.max(V))
            nV /= np.max(nV) + prec

            OV = K + nV * eps0_V

            max_O_idx = np.argsort(OV)[::-1][:sparsity]
            pull_idx = max_O_idx[0]
            push_idx = max_O_idx[1:sparsity]

            # eps
            eps = np.zeros(K.shape)
            eps[max_O_idx] = eps0 * ((np.linalg.norm(S[max_O_idx], axis=1) - 1)**2)

            S[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps[pull_idx][np.newaxis].T
            S[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps[push_idx][np.newaxis].T

            # winners
            V[pull_idx] += 1

    print(np.sum(V == 0))

    return S, V


S, UsV = train(input_SDRs=input_SDRs,
          epoch_cnt=100,
          push_delta=0.04,
          out_len=128,
          sparsity=4,
          eps0=0.0001,
          eps0_V=0.09,)
print(f'{S.max()=} {S.min()=} {np.mean(S)=}')


97
S.max()=0.3130879335872971 S.min()=-0.01963728292595758 np.mean(S)=0.005434943333546615


In [35]:
def infer_one(S: ndarray, inp_SDR: ndarray, sparsity=4) -> tuple[ndarray, ndarray]:
    K = np.dot(S, inp_SDR)

    max_O_idx = np.argsort(K)[::-1][:sparsity]

    BK = np.zeros(K.shape)
    BK[max_O_idx] = 1

    return K, BK

for inp_SDR in input_SDRs:
    O, BK = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

    print(BK.astype(int).tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [36]:
prec = 1e-30

V = np.zeros((out_len, column_size))

def train_predict(input_SDRs: list[ndarray],
                  S: ndarray,
                  V: ndarray,
                  start_I: ndarray,
                  out_len,
                  column_size,
                  prec=1e-30,
                  epoch_cnt=1,
                  sparsity=4,):
    I = np.zeros((out_len, column_size))
    O = np.zeros((out_len, column_size))

    W = np.random.rand(out_len, column_size, out_len * column_size) * 1e-2

    eps0 = 0.003

    eps0_V = 0.09

    for epoch_idx in range(epoch_cnt):
        I = np.copy(start_I)

        for inp_SDR in input_SDRs[1:]:
            K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

            I = I.reshape(out_len * column_size)

            O = np.dot(W, I)

            O /= np.max(np.abs(O)) + prec

            nV = np.abs(V - np.max(V, axis=1)[np.newaxis].T)
            nV /= np.max(nV, axis=1)[np.newaxis].T + prec

            OV = O + nV * eps0_V

            OV *= SDR[np.newaxis].T
            max_O = np.max(OV, axis=1)

            O *= SDR[np.newaxis].T
            O[OV < max_O[np.newaxis].T] = 0
            max_O_idx = np.where(O != 0)

            W[max_O_idx] += (I - W[max_O_idx] * O[max_O_idx][..., np.newaxis]) * eps0

            # winners
            V[max_O_idx] += 1

            # recur
            O[O != 0] = 1

            I = np.copy(O)

    # O.shape
    print(np.mean(V, axis=1).tolist())
    print(np.sum(V == 0))
    print(np.sum((V > 0), axis=1).tolist())
    # for col in np.rot90(V):
    #     print(f'{col.astype(int).tolist()}')

    return W, V


W, V = train_predict(input_SDRs=input_SDRs,
                     S=S,
                     V=V,
                     start_I=start_I,
                     epoch_cnt=1,
                     prec=prec,
                     out_len=out_len,
                     column_size=column_size,
                     sparsity=sparsity,)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 2.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.5, 1.0, 3.375, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1875, 0.0625, 2.5, 1.9375, 5.5625, 0.0, 0.0, 0.0, 0.0, 3.4375, 0.0, 0.0, 2.4375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5625, 4.8125, 13.4375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.4375, 0.0, 0.0, 9.1875, 7.25, 0.0, 0.0, 0.0, 0.0, 1.8125, 13.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0625, 1.0, 0.0, 7.75, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.5625, 0.0, 0.1875, 0.0, 13.75, 0.0, 0.0, 0.0, 0.0]
1705
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 1, 8, 12, 16, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 15, 11, 16, 0, 0, 0, 0, 16, 0, 0, 14, 0, 0, 0, 0, 0, 0, 8, 15, 16, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 

In [37]:
# test

def test_predict(S: ndarray, W: ndarray, input_SDRs: list[ndarray], start_I: ndarray, sparsity=4):
    error = []

    I = np.copy(start_I)

    for inp_SDR in input_SDRs[1:]:
        K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

        I = I.reshape(out_len * column_size)

        O = np.dot(W, I)

        max_O = np.max(O, axis=1)
        argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

        SDR_O = np.zeros(O.shape[0])
        SDR_O[argmax_O_idx] = 1

        err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
        # print(f'{err=}')
        error.append(err)

        O *= SDR[np.newaxis].T
        max_O = np.max(O, axis=1)
        O[O < max_O[np.newaxis].T] = 0

        # recur
        O[O != 0] = 1

        I = np.copy(O)

    error = np.array(error)
    mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
    print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

    return error


test_predict(S=S,
             W=W,
             input_SDRs=input_SDRs,
             start_I=start_I,
             sparsity=sparsity,)

# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')


np.mean(error)=2.7984031936127742 error.max()=8 error.min()=0 mean_percent=34


array([0, 2, 2, 4, 2, 6, 2, 2, 4, 2, 2, 6, 4, 2, 4, 2, 0, 6, 0, 0, 4, 4,
       2, 0, 4, 0, 0, 4, 2, 0, 4, 0, 0, 2, 2, 4, 0, 0, 6, 4, 4, 8, 6, 6,
       6, 4, 4, 2, 4, 6, 0, 0, 6, 2, 2, 6, 2, 0, 6, 8, 8, 6, 0, 8, 8, 4,
       6, 4, 6, 2, 2, 0, 2, 0, 0, 6, 6, 4, 6, 6, 2, 4, 6, 2, 6, 4, 6, 4,
       8, 2, 8, 8, 8, 4, 0, 6, 2, 8, 6, 6, 8, 2, 4, 8, 2, 8, 4, 6, 8, 4,
       4, 4, 8, 4, 6, 8, 2, 6, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 2, 0, 2,
       0, 0, 2, 0, 0, 4, 2, 6, 0, 0, 2, 4, 8, 6, 4, 6, 6, 2, 6, 6, 2, 2,
       6, 4, 4, 8, 8, 2, 6, 0, 4, 2, 8, 2, 6, 0, 4, 6, 4, 6, 4, 8, 6, 2,
       4, 2, 2, 4, 0, 0, 4, 0, 4, 0, 0, 0, 0, 2, 6, 0, 4, 4, 4, 0, 4, 6,
       6, 6, 8, 0, 2, 6, 2, 6, 6, 6, 6, 2, 6, 2, 6, 0, 2, 4, 0, 2, 2, 0,
       0, 2, 2, 0, 4, 4, 2, 4, 2, 0, 4, 0, 4, 0, 2, 2, 4, 4, 2, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 4, 0, 0, 4, 4, 6, 0, 4,
       6, 8, 4, 6, 2, 6, 0, 0, 8, 2, 6, 6, 2, 2, 6, 8, 4, 0, 0, 2, 0, 0,
       0, 6, 4, 4, 4, 2, 8, 8, 8, 6, 6, 8, 4, 4, 4,

In [38]:
def after_train_one(K: ndarray,
                    S: ndarray,
                    dS: ndarray,
                    V: ndarray,
                    I: ndarray,
                    push_delta=0.04,
                    eps0=0.0001,
                    sparsity=4,
                    eps0_V=0.09,):
    K /= np.max(np.abs(K)) + prec

    nV = np.abs(V - np.max(V))
    nV /= np.max(nV) + prec

    OV = K + nV * eps0_V

    max_O_idx = np.argsort(OV)[::-1][:sparsity]
    pull_idx = max_O_idx[0]
    push_idx = max_O_idx[1:sparsity]

    # eps
    eps = np.zeros(K.shape)
    eps[max_O_idx] = eps0 * ((np.linalg.norm(S[max_O_idx], axis=1) - 1)**2)

    dS[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps[pull_idx][np.newaxis].T
    dS[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps[push_idx][np.newaxis].T

    # winners
    V[pull_idx] += 1

    return dS


In [49]:
# after train
dS = np.zeros(S.shape)
aW = np.copy(W)
aV = np.copy(V)
aUsV = np.copy(UsV)

I = np.copy(start_I)
prev_SDR = np.copy(start_SDR)
prev_K, k_SDR = infer_one(S=S, inp_SDR=prev_SDR, sparsity=sparsity)
prev_K /= np.max(np.abs(prev_K)) + prec

preference_feedback = 0.5

for inp_SDR in input_SDRs[1:]:
    K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

    I = I.reshape(out_len * column_size)

    O = np.dot(aW, I)

    max_O = np.max(O, axis=1)
    max_O /= np.max(np.abs(max_O)) + prec
    K /= np.max(np.abs(K)) + prec
    aK = K + max_O * preference_feedback

    # clustering train
    dS = after_train_one(K=aK, S=S, dS=dS, V=aUsV, I=inp_SDR, eps0=0.0001, sparsity=sparsity)

    # next I
    O *= SDR[np.newaxis].T
    max_O = np.max(O, axis=1)
    O[O < max_O[np.newaxis].T] = 0

    O[O != 0] = 1

    # reverse input
    rvs_I = np.dot(aW.T.reshape(aW.shape[0], aW.shape[1], aW.shape[2]), O.flatten())
    rvs_I = np.max(rvs_I, axis=1)
    # print(f'{rvs_I.shape=}')
    print(f'{rvs_I.max()=} {rvs_I.min()=} {np.mean(rvs_I)=}')

    rvs_I /= np.max(np.abs(rvs_I)) + prec
    prev_K += rvs_I * preference_feedback
    
    # prev clustering train
    dS = after_train_one(K=prev_K, S=S, dS=dS, V=aUsV, I=prev_SDR, eps0=0.0001, sparsity=sparsity)

    # recur
    # O[O != 0] = 1

    I = np.copy(O)

    prev_SDR = np.copy(inp_SDR)
    prev_K = np.copy(K)


rvs_I.max()=0.04666136644654668 rvs_I.min()=0.023599704939458115 np.mean(rvs_I)=0.029946882341528064
rvs_I.max()=0.043135241331962185 rvs_I.min()=0.0229753939836431 np.mean(rvs_I)=0.030263401081494737
rvs_I.max()=0.03810490840765732 rvs_I.min()=0.023652045883412555 np.mean(rvs_I)=0.030086267688853335
rvs_I.max()=0.04507039343977284 rvs_I.min()=0.021697215174213615 np.mean(rvs_I)=0.029958044317415715
rvs_I.max()=0.0521315742599773 rvs_I.min()=0.024841973350655466 np.mean(rvs_I)=0.029974563303856464
rvs_I.max()=0.0367447229553145 rvs_I.min()=0.022256621777649963 np.mean(rvs_I)=0.029982240579663436
rvs_I.max()=0.03709732579138166 rvs_I.min()=0.02392034432900248 np.mean(rvs_I)=0.030674738991147282
rvs_I.max()=0.03671788980353653 rvs_I.min()=0.02321889764052128 np.mean(rvs_I)=0.029878304895584143
rvs_I.max()=0.035598280674805985 rvs_I.min()=0.022556752818393738 np.mean(rvs_I)=0.029964325742024737
rvs_I.max()=0.037837348080756825 rvs_I.min()=0.023515415137262134 np.mean(rvs_I)=0.029960092581

In [40]:
w = np.arange(3*2*6).reshape(3, 2, 6)
print('w', w)
print()
print('w.T', w.reshape(3, 2, 3, 2).transpose(2, 3, 0, 1).reshape(3, 2, 6))
print('w.R')
print(w.T.reshape(3, 2, 6))


w [[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]]

 [[12 13 14 15 16 17]
  [18 19 20 21 22 23]]

 [[24 25 26 27 28 29]
  [30 31 32 33 34 35]]]

w.T [[[ 0  6 12 18 24 30]
  [ 1  7 13 19 25 31]]

 [[ 2  8 14 20 26 32]
  [ 3  9 15 21 27 33]]

 [[ 4 10 16 22 28 34]
  [ 5 11 17 23 29 35]]]
w.R
[[[ 0 12 24  6 18 30]
  [ 1 13 25  7 19 31]]

 [[ 2 14 26  8 20 32]
  [ 3 15 27  9 21 33]]

 [[ 4 16 28 10 22 34]
  [ 5 17 29 11 23 35]]]


In [41]:
aW, aV = train_predict(input_SDRs=input_SDRs,
                       S=(S + dS),
                       V=aV,
                       start_I=start_I,
                       epoch_cnt=1,
                       prec=prec,
                       out_len=out_len,
                       column_size=column_size,
                       sparsity=sparsity,)

test_predict(S=(S + dS),
             W=aW,
             input_SDRs=input_SDRs,
             start_I=start_I,
             sparsity=sparsity,)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 5.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 1.0, 2.0, 6.75, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.125, 5.0, 3.875, 11.125, 0.0, 0.0, 0.0, 0.0, 6.875, 0.0, 0.0, 4.875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.125, 9.625, 26.875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.875, 0.0, 0.0, 18.375, 14.5, 0.0, 0.0, 0.0, 0.0, 3.625, 27.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.125, 2.0, 0.0, 15.5, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 25.125, 0.0, 0.375, 0.0, 27.5, 0.0, 0.0, 0.0, 0.0]
1661
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 2, 12, 16, 16, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 2, 16, 15, 16, 0, 0, 0, 0, 16, 0, 0, 15, 0, 0, 0, 0, 0, 0, 13, 16, 16, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 16, 16, 0, 0, 0, 

array([0, 0, 2, 6, 2, 2, 6, 0, 2, 6, 4, 8, 8, 8, 6, 6, 6, 8, 4, 2, 4, 4,
       4, 2, 6, 6, 8, 6, 6, 2, 0, 0, 2, 0, 0, 0, 0, 0, 6, 2, 4, 2, 4, 2,
       2, 4, 0, 0, 0, 4, 0, 4, 4, 0, 2, 4, 2, 0, 6, 2, 6, 2, 4, 4, 6, 0,
       6, 0, 0, 2, 4, 4, 6, 4, 4, 0, 4, 4, 4, 6, 2, 6, 6, 2, 4, 4, 6, 4,
       6, 6, 8, 2, 4, 2, 0, 6, 6, 4, 4, 6, 8, 4, 8, 4, 6, 4, 6, 4, 4, 2,
       4, 2, 8, 0, 6, 8, 2, 2, 4, 2, 4, 0, 2, 2, 2, 0, 6, 0, 4, 2, 6, 2,
       8, 6, 8, 4, 2, 4, 4, 2, 0, 2, 4, 0, 4, 2, 0, 4, 4, 2, 2, 0, 0, 0,
       4, 4, 4, 2, 0, 4, 6, 0, 2, 0, 6, 0, 8, 2, 4, 8, 4, 4, 4, 8, 4, 8,
       8, 8, 4, 4, 4, 2, 4, 4, 6, 8, 8, 4, 8, 2, 6, 4, 6, 2, 6, 2, 4, 6,
       8, 4, 4, 0, 4, 4, 2, 8, 2, 6, 2, 4, 6, 0, 8, 0, 0, 6, 0, 0, 2, 0,
       0, 2, 4, 0, 0, 8, 2, 4, 4, 0, 4, 0, 4, 2, 2, 6, 0, 2, 2, 2, 4, 4,
       2, 0, 2, 0, 2, 4, 0, 2, 2, 2, 0, 6, 0, 8, 2, 2, 4, 8, 8, 6, 2, 8,
       8, 8, 4, 8, 2, 8, 2, 6, 6, 2, 4, 6, 6, 0, 2, 0, 6, 2, 0, 8, 2, 0,
       8, 4, 2, 6, 4, 2, 8, 6, 8, 8, 2, 4, 0, 2, 0,